In [ ]:
# %%capture --no-stderr
# !pip install git+https://github.com/openai/CLIP timm

In [ ]:
import torch
import numpy as np
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
from tqdm import tqdm

from omegaconf import OmegaConf
from igniter.builder import build_engine
from igniter.main import get_full_config

from fsl.models.meta_arch import build_sam_fsod
from fsl.models.meta_arch import *

plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
config_file = '../configs/devit/resnet_trainval_30shot.yaml'
# config_file = '../configs/devit/devit_dinov2_trainval_xshot.yaml'

cfg = get_full_config(config_file)
# cfg = OmegaConf.load(config_file)
# print(OmegaConf.to_yaml(cfg))

OmegaConf.set_struct(cfg, False)

cfg.datasets.dataloader.batch_size = 1
cfg.datasets.dataloader.num_workers = 1
cfg.datasets.dataloader.shuffle=False
cfg.options.train = False
cfg.options.eval = True

# cfg.models.devit_dinov2_fsod.prototype_file="/root/krishneel/Downloads/fsl/prototypes/devit_dinov2_fsod/fs_coco_trainval_novel_30shot.pkl"
# cfg.models.devit_dinov2_fsod.background_prototype_file="/root/krishneel/Downloads/fsl/prototypes/devit_dinov2_fsod/panoptic_train2017_only_stuffs.pkl"
# cfg.models.devit_dinov2_fsod.background_prototype_file=None

cfg.build.resnet_fsod.weights="s3://sr-shokunin/perception/models/fsl/resnet_fsod_coco17_30shot/model_0000020.pt"
# cfg.build.sam_fsod.weights = "s3://sr-shokunin/perception/models/fsl/sam_vitb_coco17_30shot/model_0000005.pt"
# cfg.build.dinov2_fsod.weights = "s3://sr-shokunin/perception/models/fsl/dinov2_vitb_coco17_5shot/model_0000009.pt"

# cfg.build.devit_dinov2_fsod.weights = "s3://sr-shokunin/perception/models/fsl/devit_dinov2/model_0000005.pt"
# cfg.build.model = "dinov2_fsod"
cfg.build.resnet_fsod.mode = "val"

engine = build_engine(cfg)
print(OmegaConf.to_yaml(cfg.build))

In [ ]:
%%capture --no-stderr
engine._model.eval().to(torch.float16)

# print(engine._model.classifier.bg_tokens.shape)

In [ ]:
size = max([len(iid) for iid in engine._model.classifier._all_cids]) + 1
gt_labels = lambda indices: [engine._model.classifier._all_cids[i] for i in indices]

def accuracy(scores, gt_names, topk = 1):
    _, indices = torch.topk(scores, topk, dim=1)
    pred_labels = [gt_labels(index) for index in indices]

    count = 0.0
    for gt_name, labels in zip(gt_names, pred_labels):
        count += 1 if gt_name in labels else 0
    return count / len(gt_names)

def print_labels(scores, targets, k = 2):
    gt_names = targets[0]['gt_proposal'].labels
    _, indexes = torch.topk(scores, k, dim=1)
    
    for name, i in zip(gt_names, indexes.cpu().numpy()):
        string = name.ljust(size) + " | " + "".join([n.ljust(size) for n in gt_labels(i)])
        print(string)

In [ ]:
acc_top1, acc_top5 = 0.0, 0.0
progress_bar = tqdm(total=len(engine._dataloader), desc="Processing", position=0, leave=True)

for k, (images, targets) in enumerate(engine._dataloader, 1):
    with torch.no_grad():
        output, _ = engine._model(images, targets)
    scores = output['scores']
    # gt_names = targets[0]['gt_proposal'].labels
    gt_names = [target['gt_proposal'].labels for target in targets]
    gt_names = [item for sublist in gt_names for item in sublist]

    t1 = accuracy(scores, gt_names, topk=1)
    t5 = accuracy(scores, gt_names, topk=5)

    acc_top1 += (t1 - acc_top1) / k
    acc_top5 += (t5 - acc_top5) / k

    # print("\nImage ID: ", targets[0]['gt_proposal'].image_id)
    # print_labels(scores, targets, k=5)
    progress_bar.set_description(f"Top 1: {acc_top1:.3f} | Top 5 {acc_top5:.3f}")
    progress_bar.update(1)

progress_bar.close()

print("\nTop 1: ", acc_top1, "Top 5: ", acc_top5)

In [ ]:
indices = scores.argmax(1).cpu().numpy().tolist()
print(gt_labels(indices), gt_names)

In [ ]:
engine._model.classifier._all_cids

In [ ]:
from fsl.datasets.s3_coco_dataset import *
from igniter.registry import dataset_registry
from igniter.builder import build_dataloader
from omegaconf import OmegaConf
from igniter.builder import build_engine
from igniter.main import get_full_config
import matplotlib.pyplot as plt
import cv2 as cv

c = '../configs/devit/prototypes/foreground_prototypes.yaml'
cfg = get_full_config(c)
cfg.build.model = 'devit_dinov2_fsod'
cfg.datasets.dataloader.shuffle = True

print(OmegaConf.to_yaml(cfg.build))

dl = build_dataloader(cfg, 'val')

In [ ]:
for i, (image, target) in enumerate(dl):

    print(target)
    
    im = image[0].permute(1, 2, 0).cpu().numpy()
    im = cv.cvtColor(im, cv.COLOR_RGB2BGR)
    im = cv.cvtColor(im, cv.COLOR_RGB2BGR)
    
    for bb in target[0]['bboxes']:
        x1, y1, x2, y2 = bb.int().cpu().numpy()
        cv.rectangle(im, (x1, y1), (x2, y2), (0, 255, 0), 3)

    plt.imshow(im); plt.show()

    if i > 20:
        break

# Visualization

In [ ]:
from sklearn.manifold import TSNE

def plot_tsne(prototype):
    high_dim_data = prototype.normalized_embedding.cpu().numpy()
    tsne = TSNE(n_components=3, random_state=256)  # 3 components for 3D visualization
    low_dim_data = tsne.fit_transform(high_dim_data)

    # Initialize t-SNE model and fit_transform the high-dimensional data
    tsne = TSNE(n_components=2, random_state=42)  # 2 components for 2D visualization
    low_dim_data = tsne.fit_transform(high_dim_data)

    # Plot the 2D t-SNE representation
    # plt.figure(figsize=(8, 6))
    plt.scatter(low_dim_data[:, 0], low_dim_data[:, 1], marker='o', c='b', edgecolors='k')
    plt.title('t-SNE Visualization of High-dimensional Data')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.grid(True)
    plt.show()


def plot_distance(prototype):
    data = prototype.normalized_embedding.float().cpu()
    labels = prototype.labels

    indices = np.argsort(labels)
    labels = np.array(labels)[indices]
    data = data[indices]

    # Calculate pairwise distances using Euclidean distance for demonstration
    expanded_data = data.unsqueeze(1)  # Add a new dimension to make broadcasting work
    diff = expanded_data - data
    dist_matrix = torch.norm(diff, dim=2)  # Compute Euclidean distances

    # Convert PyTorch tensor to NumPy for visualization (if needed)
    dist_matrix_np = dist_matrix.numpy()

    # Plotting the distance matrix
    plt.figure(figsize=(16, 12))
    plt.imshow(dist_matrix_np, cmap='viridis', interpolation='nearest')
    plt.colorbar(label='Distance')
    plt.title('Distance Matrix')
    plt.xticks(ticks=np.arange(len(labels)), labels=labels, rotation=60, ha='right', fontsize=8)  # Replace legends for x-axis
    plt.yticks(ticks=np.arange(len(labels)), labels=labels, fontsize=8)
    plt.xlabel('Data Index')
    plt.ylabel('Data Index')
    plt.show()

In [ ]:
from fsl.utils import ProtoTypes

In [ ]:
from fsl.tools.prototype_builder import _post_process_prototypes

p = _post_process_prototypes('/root/krishneel/Downloads/fsl/prototypes/devit_dinov2_fsod/', 'bg_test.pkl', clean=False, reduction='per_class_cluster', cluster_size=10)

print(p.embeddings.shape)

In [ ]:
pt1 = ProtoTypes.load('/root/krishneel/Downloads/fsl/prototypes/devit_dinov2_fsod/bg_test.pkl')
pt2 = ProtoTypes.load('/root/krishneel/Downloads/fsl/prototypes/dinov2/coco14/background_prototypes_vitb14.pkl')

print(pt1.embeddings.shape, pt2.embeddings.shape)

plot_tsne(pt1)
plot_tsne(pt2)

plot_distance(pt1)
plot_distance(pt2)